In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment_simulator.models import (
    SimulatedStock, 
    SimulatedStockCovariance, 
    SimulatedStockBuffer,
    SimulatedSector
)

import numpy as np
import json
from datetime import datetime

In [2]:
old_sectors_list = SimulatedSector.objects.all()
old_sectors = {}
for an_old_sector in old_sectors_list:
    old_sectors[an_old_sector.name] = an_old_sector.name

new_sectors_list = []
sector_names = [
    "Healthcare", 
    "Materials", 
    "Real Estate", 
    "Consumer Staples",
    "Consumer Discretionary",
    "Utilities",
    "Energy",
    "Industrials",
    "Consumer Services",
    "Financials",
    "Technology"
]

for s in range(len(sector_names)):
    if sector_names[s] not in old_sectors:
        new_sectors_list.append(
            SimulatedSector(
                name = sector_names[s],
                index = s+1
            )
        )

SimulatedSector.objects.bulk_create(new_sectors_list)

[]

In [3]:
np.random.seed(0)
new_sectors = SimulatedSector.objects.all()
new_sectors = list(new_sectors)
no_of_new_sectors = len(new_sectors)
SimulatedStock.objects.all().delete()
nsew_stocks = []
for i in range(1,6):
    avg_price = np.random.choice(list(range(1,11)))*np.random.choice(list(range(1,11)))
    grad_temp = avg_price/100
    sd = np.random.choice([grad_temp*2, grad_temp/2])
    pos_gradient = np.random.normal(grad_temp, sd)
    gradient = np.random.choice([pos_gradient, -1*pos_gradient])
    sector_index = i%no_of_new_sectors
    min_price = avg_price/np.random.choice([3,4,5,6])
    max_price = avg_price*np.random.choice([3,4,5,6])
    avg_forward_steps = np.random.choice(list(range(7,18)))
    avg_backward_steps = avg_forward_steps
    volume_sd=np.random.rand()*np.random.choice(list(range(500,2500,250)))
    nsew_stocks.append(
        SimulatedStock(
            name = f"stock_{i}",
            index = i,
            sector = new_sectors[sector_index],
            price_gradient = gradient,
            price_sd = sd,
            min_price = min_price,
            max_price = max_price,
            avg_forward_steps = avg_forward_steps,
            avg_backward_steps = avg_backward_steps,
            price_steps_left = avg_forward_steps,
            volume_sd = volume_sd,
            price_x_volume_factor = np.random.rand()*np.random.choice(list(range(1,6)))/7,
            volume_x_price_factor = np.random.rand()*np.random.choice(list(range(1,6)))/7
        )
    )
SimulatedStock.objects.bulk_create(nsew_stocks)

[<SimulatedStock: 316, 1, stock_1>,
 <SimulatedStock: 317, 2, stock_2>,
 <SimulatedStock: 318, 3, stock_3>,
 <SimulatedStock: 319, 4, stock_4>,
 <SimulatedStock: 320, 5, stock_5>]

In [4]:
np.random.seed(0)

SimulatedStockCovariance.objects.all().delete()
comvars = []
accounted_for_stocks = {}
new_stocks = SimulatedStock.objects.all()
for nw_stck in new_stocks:
    # print("nw_stck.id", nw_stck.id)
    # print("recheck", json.dumps(accounted_for_stocks, indent=3))
    if nw_stck.id not in accounted_for_stocks:
        accounted_for_stocks[nw_stck.id] = {}
    stck_a_stcks = accounted_for_stocks[nw_stck.id]
    for a_stck in new_stocks:
        if a_stck.id != nw_stck.id:
            if a_stck.id not in accounted_for_stocks:
                accounted_for_stocks[a_stck.id] = {}
            stck_b_stcks = accounted_for_stocks[a_stck.id]
            if (
                a_stck.id not in stck_a_stcks
                and nw_stck.id not in stck_b_stcks
            ):
                stck_a_stcks[a_stck.id] = a_stck.id
                stck_b_stcks[nw_stck.id] = nw_stck.id
                new_covar = SimulatedStockCovariance(
                    stock_a = nw_stck,
                    stock_b = a_stck,
                    factor = np.random.rand()*np.random.choice([0.5,-0.5])
                )
                comvars.append(new_covar)
                # print(json.dumps(accounted_for_stocks, indent=3))
                # print("")

SimulatedStockCovariance.objects.bulk_create(comvars)

[<SimulatedStockCovariance: 14881, stock_1, stock_2>,
 <SimulatedStockCovariance: 14882, stock_1, stock_3>,
 <SimulatedStockCovariance: 14883, stock_1, stock_4>,
 <SimulatedStockCovariance: 14884, stock_1, stock_5>,
 <SimulatedStockCovariance: 14885, stock_2, stock_3>,
 <SimulatedStockCovariance: 14886, stock_2, stock_4>,
 <SimulatedStockCovariance: 14887, stock_2, stock_5>,
 <SimulatedStockCovariance: 14888, stock_3, stock_4>,
 <SimulatedStockCovariance: 14889, stock_3, stock_5>,
 <SimulatedStockCovariance: 14890, stock_4, stock_5>]

In [5]:
starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

SimulatedStockBuffer.objects.all().delete()
z_stocks = SimulatedStock.objects.all()
buffer = []
for z_stck in z_stocks:
    mid_price = (z_stck.min_price + z_stck.max_price)
    mid_price = mid_price/2
    starting_snapshot = SimulatedStockBuffer(
        stock = z_stck,
        captured_at = starting_time_step,
        price_snapshot = mid_price,
        ldcp = mid_price,
        open = mid_price,
        high = mid_price,
        low = mid_price,
        change = 0,
        volume = 0
    )
    buffer.append(starting_snapshot)

SimulatedStockBuffer.objects.bulk_create(buffer)

/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTimeField SimulatedStockBuffer.captured_at received a naive datetime (1995-01-01 10:00:00) while time zone support is active.
  warnings.warn(


[<SimulatedStockBuffer: 906, stock_1, 15.75>,
 <SimulatedStockBuffer: 907, stock_2, 26.0>,
 <SimulatedStockBuffer: 908, stock_3, 17.0>,
 <SimulatedStockBuffer: 909, stock_4, 52.0>,
 <SimulatedStockBuffer: 910, stock_5, 28.5>]

In [7]:
updtd_stcks = []
u_stocks = SimulatedStock.objects.all()
for u_stck in u_stocks:
    u_stck.price_steps_left = u_stck.avg_forward_steps
    updtd_stcks.append(u_stck)

u_stocks.bulk_update(updtd_stcks, ["price_steps_left"])

5

In [6]:
SimulatedStockBuffer.objects.all().delete()

(6660, {'environment_simulator.SimulatedStockBuffer': 6660})